<a href="https://colab.research.google.com/github/corinnakeum1/DS-4002/blob/main/APIHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import pandas as pd
import requests

# Get stock symbols from the user
stocks = input("Enter stock symbols separated by commas: ")

# Replace with your actual API key
apikey = "9vw56mDZnY4DXUvuTfeZT72Em39DKeJf70ViwvUL" # @param {"type":"string","placeholder":"9vw56mDZnY4DXUvuTfeZT72Em39DKeJf70ViwvUL"}
url = "https://yfapi.net/v6/finance/quote"
querystring = {"symbols": stocks}
headers = {
    'x-api-key': apikey
}

try:
    response = requests.get(url, headers=headers, params=querystring)
    response.raise_for_status()  # Raise an exception for bad status codes

    data = response.json()

    if 'quoteResponse' in data and 'result' in data['quoteResponse']:
      df = pd.DataFrame(data['quoteResponse']['result'])

      if not df.empty:
        # Display basic information
        for index, row in df.iterrows():
          print(f"Stock Ticker: {row['symbol']}, Company: {row['longName']}, Current Market Price: ${row['regularMarketPrice']:.2f}")


        # Task 2.1: Fetch additional data
        modules = ["summaryDetail"] # Add more modules as needed.
        module_choice = input(f"Choose a module from {modules} to fetch additional data or press enter to skip: ")

        if module_choice in modules:
          url = f"https://yfapi.net/v11/finance/quoteSummary/{stocks}"
          querystring = {"modules": module_choice}
          response = requests.get(url, headers=headers, params=querystring)
          response.raise_for_status()
          data = response.json()
          if module_choice in data['quoteSummary']['result'][0]:
              summary_df = pd.DataFrame([data['quoteSummary']['result'][0][module_choice]])
              print("\nAdditional Data:")
              print(summary_df)
          else:
              print(f"Module '{module_choice}' data not found in the response.")
        else:
          print("Invalid module chosen. Skipping additional data retrieval.")

      else:
          print("No data found for the specified stocks.")
    else:
      print("Unexpected response format from API.")
      print(json.dumps(data, indent=4)) # print the json response for debuging

except requests.exceptions.RequestException as e:
  print(f"An error occurred during the API request: {e}")
except KeyError as e:
  print(f"An error occurred accessing the data: {e}")
except Exception as e:
  print(f"An unexpected error occurred: {e}")


Enter stock symbols separated by commas: TSLA,NVDA
Stock Ticker: TSLA, Company: Tesla, Inc., Current Market Price: $290.80
Stock Ticker: NVDA, Company: NVIDIA Corporation, Current Market Price: $131.28
Choose a module from ['summaryDetail'] to fetch additional data or press enter to skip: 
Invalid module chosen. Skipping additional data retrieval.


In [23]:
import json
import pandas as pd
import requests

# Get stock symbols from the user
stocks = input("Enter stock symbols separated by commas: ").strip()

# Replace with your actual API key
apikey = "9vw56mDZnY4DXUvuTfeZT72Em39DKeJf70ViwvUL"  # Use environment variables for security

# Yahoo Finance API Headers
headers = {'x-api-key': apikey}

# Available modules
modules = ["summaryDetail"]
module_choice = input(f"Choose a module from {modules} to fetch additional data or press enter to skip: ").strip()

# Validate user choice
if module_choice in modules:
    summary_df = pd.DataFrame()  # Initialize an empty DataFrame

    for symbol in stocks.split(','):
        symbol = symbol.strip()
        url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
        querystring = {"modules": module_choice}

        try:
            response = requests.get(url, headers=headers, params=querystring)
            response.raise_for_status()
            data = response.json()

            # Check if response contains data
            if module_choice in data.get('quoteSummary', {}).get('result', [{}])[0]:
                summary_detail = data['quoteSummary']['result'][0][module_choice]

                # Extract required fields safely
                fifty_two_week_high = summary_detail.get('fiftyTwoWeekHigh', {}).get('raw', 'N/A')
                fifty_two_week_low = summary_detail.get('fiftyTwoWeekLow', {}).get('raw', 'N/A')
                roa = summary_detail.get('returnOnAssets', {}).get('raw', 'N/A')

                # Create a DataFrame for the current stock
                symbol_df = pd.DataFrame({
                    'Symbol': [symbol],
                    '52 Week High': [fifty_two_week_high],
                    '52 Week Low': [fifty_two_week_low],
                    'Return on Assets (ROA)': [roa]
                })

                summary_df = pd.concat([summary_df, symbol_df], ignore_index=True)

            else:
                print(f"Module '{module_choice}' data not found for {symbol}.")

        except (KeyError, IndexError, TypeError, json.JSONDecodeError) as e:
            print(f"Error processing data for {symbol}: {e}")
            print(json.dumps(data, indent=4))  # Debugging output

    # Display results
    if not summary_df.empty:
        print("\nAdditional Data Retrieved:")
        print(summary_df)
    else:
        print("No additional data retrieved.")

else:
    print("Invalid module chosen or skipped additional data retrieval.")



Enter stock symbols separated by commas: TLSA
Choose a module from ['summaryDetail'] to fetch additional data or press enter to skip: summaryDetail

Additional Data Retrieved:
  Symbol  52 Week High  52 Week Low Return on Assets (ROA)
0   TLSA          1.74         0.41                    N/A


In [24]:
# Replace with your actual API key
apikey = "9vw56mDZnY4DXUvuTfeZT72Em39DKeJf70ViwvUL"  # Secure storage recommended

# Yahoo Finance API Headers
headers = {'x-api-key': apikey}

# Step 1: Get Trending Stocks
trending_url = "https://yfapi.net/v1/finance/trending/US"  # Trending in US
try:
    response = requests.get(trending_url, headers=headers)
    response.raise_for_status()
    trending_data = response.json()

    # Extract tickers from response
    trending_tickers = [item["symbol"] for item in trending_data.get("finance", {}).get("result", [{}])[0].get("quotes", [])]

    if not trending_tickers:
        print("No trending stocks found.")
    else:
        print(f"Trending Stocks: {', '.join(trending_tickers)}")

        # Step 2: Get stock details
        stock_details_url = "https://yfapi.net/v6/finance/quote"
        querystring = {"symbols": ",".join(trending_tickers)}

        response = requests.get(stock_details_url, headers=headers, params=querystring)
        response.raise_for_status()
        stock_data = response.json()

        # Extract relevant stock details
        stock_list = []
        for stock in stock_data.get("quoteResponse", {}).get("result", []):
            symbol = stock.get("symbol", "N/A")
            name = stock.get("longName", "N/A")
            price = stock.get("regularMarketPrice", "N/A")
            fifty_two_week_high = stock.get("fiftyTwoWeekHigh", "N/A")
            fifty_two_week_low = stock.get("fiftyTwoWeekLow", "N/A")

            stock_list.append({
                "Ticker": symbol,
                "Name": name,
                "Current Price": price,
                "52 Week High": fifty_two_week_high,
                "52 Week Low": fifty_two_week_low
            })

        # Convert to Pandas DataFrame
        trending_df = pd.DataFrame(stock_list)
        print("\nTrending Stocks Data:")
        print(trending_df)

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the API request: {e}")
except KeyError as e:
    print(f"An error occurred accessing the data: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Trending Stocks: NVDA, SNOW, CRM, IONQ, BYND, MRNA, AI, TDOC, MARA, APP, NIO, EBAY, NTNX, AVGO, ROOT, SNPS, MULN, PSTG, JOBY, ODD

Trending Stocks Data:
   Ticker                     Name  Current Price  52 Week High  52 Week Low
0    NVDA       NVIDIA Corporation         131.28       153.130       75.606
1    SNOW           Snowflake Inc.         166.19       233.880      107.130
2     CRM         Salesforce, Inc.         307.33       369.000      212.000
3    IONQ               IonQ, Inc.          29.93        54.740        6.220
4    BYND        Beyond Meat, Inc.           3.56        12.120        3.300
5    MRNA            Moderna, Inc.          33.58       170.470       29.250
6      AI              C3.ai, Inc.          26.44        45.080       18.850
7    TDOC     Teladoc Health, Inc.          10.99        15.950        6.760
8    MARA      MARA Holdings, Inc.          12.45        34.090       12.050
9     APP     AppLovin Corporation         331.00       525.150       57.400
